# WRDS CRSP 데이터 수집
논문 "Re-Imaging Price Trends" 구현용 데이터 생성

In [ ]:
!pip install wrds

In [ ]:
import wrds
import pandas as pd
import numpy as np

## 1. WRDS 연결

In [ ]:
print("WRDS 연결 중...")
db = wrds.Connection()
print("연결 성공!")

WRDS 연결 중...
Enter your WRDS username [root]:nsj020816
Enter your password:··········
WRDS recommends setting up a .pgpass file.
Create .pgpass file now [y/n]?: y
Created .pgpass file successfully.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done
연결 성공!


## 2. CRSP 데이터 쿼리

In [ ]:
query = """
    SELECT
        a.permno AS code,
        a.date,
        a.prc AS close,
        a.vol AS volume,
        a.ret,          -- 수익률
        a.openprc AS open,
        a.askhi AS high,
        a.bidlo AS low,
        a.shrout,       -- 발행주식수 (시가총액 계산용)
        b.ticker
    FROM
        crsp.dsf AS a
    LEFT JOIN
        crsp.dsenames AS b
        ON a.permno = b.permno AND b.namedt <= a.date AND a.date <= b.nameendt
    WHERE
        a.date BETWEEN '1993-01-01' AND '2019-12-31'
        AND b.shrcd IN (10, 11)
        AND b.exchcd IN (1, 2, 3)
"""

print("데이터 다운로드 중... (시간 소요)")
df = db.raw_sql(query, date_cols=['date'])
db.close()

print(f"다운로드 완료: {len(df):,}개 레코드")
print(f"종목 수: {df['code'].nunique():,}")
print(f"기간: {df['date'].min()} ~ {df['date'].max()}")

데이터 다운로드 중... (시간 소요)
다운로드 완료: 34,267,180개 레코드
종목 수: 15,642
기간: 1993-01-04 00:00:00 ~ 2019-12-31 00:00:00


2.데이터 확인


In [ ]:
df.head()

,code,date,close,volume,ret,open,high,low,shrout,ticker
0,10026,2019-07-09,162.92999,74826.0,-0.002388,163.22,164.0,162.64999,18830.0,JJSF
1,10026,2019-07-10,163.64999,62022.0,0.004419,163.14,164.73,162.22,18830.0,JJSF
2,10026,2019-07-11,163.24001,42782.0,-0.002505,164.10001,164.28999,162.42999,18830.0,JJSF
3,10026,2019-07-12,163.89,60461.0,0.003982,163.63,164.63,162.0197,18830.0,JJSF
4,10026,2019-07-15,162.95,73890.0,-0.005736,163.83,164.63,162.63,18830.0,JJSF


In [ ]:
print("\n데이터 통계:")
print(df.describe())

print("\n결측치 확인:")
print(df.isnull().sum())

print("\n데이터 타입 확인:")
print(df.info())


데이터 통계:
               code                           date        close  \
count    34267180.0                       34267180   34254100.0   
mean   64654.149036  2004-08-01 01:22:32.269594496    44.303095   
min         10001.0            1993-01-04 00:00:00     -18800.0   
25%         44813.0            1997-12-17 00:00:00       4.2599   
50%         78064.0            2003-05-09 00:00:00        12.92   
75%         85271.0            2010-10-13 00:00:00       27.125   
max         93436.0            2019-12-31 00:00:00     340380.0   
std     27757.81781                            NaN  2018.918731   

               volume         ret        open         high           low  \
count      34254155.0  34243846.0  32417427.0   34254100.0    34254100.0   
mean    662043.377724    0.000881   47.346165    46.120687     45.101454   
min               0.0   -0.971698      0.0001       0.0014        0.0001   
25%            8800.0   -0.014975        5.35         5.31           5.0   
50%    

In [ ]:
df['date'] = pd.to_datetime(df['date'])
df = df.sort_values(by=['code', 'date'])

In [ ]:
df_no_open = df[df['open'].isnull()]
negative_close_no_open = df_no_open[df_no_open['close'] < 0]

if not negative_close_no_open.empty:
    print(f"'open' 값이 없는 {len(negative_close_no_open):,}개의 레코드에서 'close' 값이 음수입니다.")
    print("예시 레코드:")
    print(negative_close_no_open.head())
else:
    print("'open' 값이 없는 레코드 중 'close' 값이 음수인 경우는 없습니다.")


'open' 값이 없는 1,787,742개의 레코드에서 'close' 값이 음수입니다.
예시 레코드:
      code       date  close  volume       ret  open   high    low  shrout  \
849  10001 1993-01-05 -14.25     0.0 -0.017241  <NA>   14.5   14.0  1080.0   
850  10001 1993-01-06 -14.25     0.0       0.0  <NA>   14.5   14.0  1080.0   
854  10001 1993-01-12 -14.25     0.0 -0.017241  <NA>   14.5   14.0  1080.0   
857  10001 1993-01-15  -14.0     0.0  0.018182  <NA>  14.25  13.75  1080.0   
858  10001 1993-01-18  -14.0     0.0       0.0  <NA>  14.25  13.75  1080.0   

    ticker  
849   GFGC  
850   GFGC  
854   GFGC  
857   GFGC  
858   GFGC  


## 4. 기본 전처리

In [ ]:
# 기존 전처리는 위 셀로 이동됨

In [ ]:
print("시가총액 및 EWMA 변동성 계산 중...")

# CRSP prc는 음수일 수 있음 (bid/ask 평균 표시)
df['close'] = df['close'].abs()

# 1. 시가총액 계산 (단위: 천달러)
# Market Cap = abs(prc) * shrout * 1000
# shrout는 천주 단위이므로 1000을 곱해서 달러 단위로 변환
df['market_cap'] = df['close'] * df['shrout'] * 1000

print(f"시가총액 통계:")
print(f"  평균: ${df['market_cap'].mean():,.0f}")
print(f"  중간값: ${df['market_cap'].median():,.0f}")
print(f"  결측값: {df['market_cap'].isnull().sum():,}개")

# 2. EWMA 변동성 계산 (alpha=0.05, 1일 지연)
print("\nEWMA 변동성 계산 중...")

def calculate_ewma_vol(group_df, alpha=0.05):
    """종목별 EWMA 변동성 계산"""
    returns = group_df['ret'].copy()

    # 결측값 처리
    returns = returns.fillna(0)

    # 수익률 제곱
    ret_squared = returns ** 2

    # EWMA 계산 (pandas ewm 사용)
    ewma_vol = ret_squared.ewm(alpha=alpha, adjust=False).mean()

    # 1일 지연 (논문에서 요구사항)
    ewma_vol = ewma_vol.shift(1)

    return ewma_vol

# 종목별로 EWMA 변동성 계산
df['ewma_vol'] = df.groupby('code').apply(
    lambda x: calculate_ewma_vol(x), include_groups=False
).reset_index(level=0, drop=True)

print(f"EWMA 변동성 통계:")
print(f"  평균: {df['ewma_vol'].mean():.6f}")
print(f"  중간값: {df['ewma_vol'].median():.6f}")
print(f"  결측값: {df['ewma_vol'].isnull().sum():,}개")

# 샘플 확인
print(f"\n샘플 데이터:")
sample_df = df[['code', 'date', 'close', 'shrout', 'market_cap', 'ret', 'ewma_vol']].head(10)
print(sample_df)

시가총액 및 EWMA 변동성 계산 중...
시가총액 통계:
  평균: $2,921,413,831
  중간값: $203,044,255
  결측값: 13,080개

EWMA 변동성 계산 중...
EWMA 변동성 통계:
  평균: 0.002397
  중간값: 0.000816
  결측값: 15,642개

샘플 데이터:
      code       date  close  shrout  market_cap       ret  ewma_vol
848  10001 1993-01-04   14.5  1080.0  15660000.0  0.035714       NaN
849  10001 1993-01-05  14.25  1080.0  15390000.0 -0.017241  0.001275
850  10001 1993-01-06  14.25  1080.0  15390000.0       0.0  0.001227
851  10001 1993-01-07   14.5  1080.0  15660000.0  0.017544  0.001165
852  10001 1993-01-08   14.5  1080.0  15660000.0       0.0  0.001122
853  10001 1993-01-11   14.5  1080.0  15660000.0       0.0  0.001066
854  10001 1993-01-12  14.25  1080.0  15390000.0 -0.017241  0.001013
855  10001 1993-01-13  13.75  1080.0  14850000.0 -0.035088  0.000977
856  10001 1993-01-14  13.75  1080.0  14850000.0       0.0  0.000990
857  10001 1993-01-15   14.0  1080.0  15120000.0  0.018182  0.000940


## 4. 시가총액 및 EWMA 변동성 계산

## 5. 미래 수익률 계산

In [ ]:
print("미래 수익률 계산 중")

# 수익률 컬럼 초기화
df['ret5'] = np.nan
df['ret20'] = np.nan
df['ret60'] = np.nan

# 종목별 처리
df['ret5'] = df.groupby('code')['close'].pct_change(5).shift(-5) * 100
df['ret20'] = df.groupby('code')['close'].pct_change(20).shift(-20) * 100
df['ret60'] = df.groupby('code')['close'].pct_change(60).shift(-60) * 100

# 이진 분류 라벨 생성 (수정됨)
df['label_5'] = np.nan
df['label_20'] = np.nan
df['label_60'] = np.nan

not_na_mask5 = df['ret5'].notna()
not_na_mask20 = df['ret20'].notna()
not_na_mask60 = df['ret60'].notna()  # 변수명도 수정

# 올바른 라벨 생성 (각각 해당 수익률 기준)
df.loc[not_na_mask5, 'label_5'] = np.where(df.loc[not_na_mask5, 'ret5'] > 0, 1, 0)
df.loc[not_na_mask20, 'label_20'] = np.where(df.loc[not_na_mask20, 'ret20'] > 0, 1, 0)  # 수정: ret20 사용
df.loc[not_na_mask60, 'label_60'] = np.where(df.loc[not_na_mask60, 'ret60'] > 0, 1, 0)  # 수정: ret60 사용

print(df[['code', 'date', 'close', 'ret5', 'ret20', 'ret60', 'label_5', 'label_20', 'label_60']].head(10))

미래 수익률 계산 중


/tmp/ipython-input-10-3444644217.py:9: FutureWarning: The default fill_method='ffill' in SeriesGroupBy.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  df['ret5'] = df.groupby('code')['close'].pct_change(5).shift(-5) * 100
/tmp/ipython-input-10-3444644217.py:10: FutureWarning: The default fill_method='ffill' in SeriesGroupBy.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  df['ret20'] = df.groupby('code')['close'].pct_change(20).shift(-20) * 100
/tmp/ipython-input-10-3444644217.py:11: FutureWarning: The default fill_method='ffill' in SeriesGroupBy.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None'

      code       date  close      ret5     ret20     ret60  label_5  label_20  \
848  10001 1993-01-04   14.5       0.0 -5.172414 -2.586207      0.0       0.0   
849  10001 1993-01-05  14.25       0.0 -3.508772       0.0      0.0       0.0   
850  10001 1993-01-06  14.25 -3.508772 -1.754386 -0.877193      0.0       0.0   
851  10001 1993-01-07   14.5 -5.172414 -3.448276 -2.586207      0.0       0.0   
852  10001 1993-01-08   14.5 -3.448276 -3.448276       0.0      0.0       0.0   
853  10001 1993-01-11   14.5 -3.448276 -5.172414 -2.586207      0.0       0.0   
854  10001 1993-01-12  14.25 -3.508772       0.0 -0.877193      0.0       0.0   
855  10001 1993-01-13  13.75  3.636364       0.0       0.0      1.0       0.0   
856  10001 1993-01-14  13.75  3.636364  1.818182  2.727273      1.0       1.0   
857  10001 1993-01-15   14.0 -1.785714  1.785714  3.571429      0.0       1.0   

     label_60  
848       0.0  
849       0.0  
850       0.0  
851       0.0  
852       0.0  
853       0.

## 6. 최종 데이터 확인

In [ ]:
print("최종 데이터 현황:")
print(f"전체 레코드: {len(df):,}")
print(f"종목 수: {df['code'].nunique():,}")
print(f"기간: {df['date'].min()} ~ {df['date'].max()}")
print(f"미래수익률 유효 레코드: {df['ret5'].notna().sum():,}")

# 샘플 확인
print("\n최종 데이터 샘플:")
df.head(10)

최종 데이터 현황:
전체 레코드: 34,267,180
종목 수: 15,642
기간: 1993-01-04 00:00:00 ~ 2019-12-31 00:00:00
미래수익률 유효 레코드: 34,179,908

최종 데이터 샘플:


,code,date,close,volume,ret,open,high,low,shrout,ticker,market_cap,ewma_vol,ret5,ret20,ret60,label_5,label_20,label_60
848,10001,1993-01-04,14.5,150.0,0.035714,14.5,14.5,14.5,1080.0,GFGC,15660000.0,NaN,0.0,-5.172414,-2.586207,0.0,0.0,0.0
849,10001,1993-01-05,14.25,0.0,-0.017241,<NA>,14.5,14.0,1080.0,GFGC,15390000.0,0.001275,0.0,-3.508772,0.0,0.0,0.0,0.0
850,10001,1993-01-06,14.25,0.0,0.0,<NA>,14.5,14.0,1080.0,GFGC,15390000.0,0.001227,-3.508772,-1.754386,-0.877193,0.0,0.0,0.0
851,10001,1993-01-07,14.5,228.0,0.017544,14.5,14.5,14.5,1080.0,GFGC,15660000.0,0.001165,-5.172414,-3.448276,-2.586207,0.0,0.0,0.0
852,10001,1993-01-08,14.5,1375.0,0.0,14.25,14.5,14.25,1080.0,GFGC,15660000.0,0.001122,-3.448276,-3.448276,0.0,0.0,0.0,0.0
853,10001,1993-01-11,14.5,200.0,0.0,14.5,14.5,14.5,1080.0,GFGC,15660000.0,0.001066,-3.448276,-5.172414,-2.586207,0.0,0.0,0.0
854,10001,1993-01-12,14.25,0.0,-0.017241,<NA>,14.5,14.0,1080.0,GFGC,15390000.0,0.001013,-3.508772,0.0,-0.877193,0.0,0.0,0.0
855,10001,1993-01-13,13.75,700.0,-0.035088,14.0,14.0,13.75,1080.0,GFGC,14850000.0,0.000977,3.636364,0.0,0.0,1.0,0.0,0.0
856,10001,1993-01-14,13.75,400.0,0.0,13.75,13.75,13.75,1080.0,GFGC,14850000.0,0.000990,3.636364,1.818182,2.727273,1.0,1.0,1.0
857,10001,1993-01-15,14.0,0.0,0.018182,<NA>,14.25,13.75,1080.0,GFGC,15120000.0,0.000940,-1.785714,1.785714,3.571429,0.0,1.0,1.0


## 7. 저장

In [ ]:
TRAIN_VAL_FILENAME = 'data_1993_2000_train_val.parquet'
TEST_FILENAME = 'data_2001_2019_test.parquet'


# --- 훈련/검증 데이터 (1993-2000) ---
print("훈련/검증 데이터 (1993-2000) 분할 및 저장 중...")
train_val_df = df[df['date'].dt.year <= 2000].copy()
train_val_df.to_parquet(TRAIN_VAL_FILENAME, index=False)
print(f"저장 완료: {TRAIN_VAL_FILENAME} ({len(train_val_df):,} 레코드)")

# --- 테스트 데이터 (2001-2019) ---
print("테스트 데이터 (2001-2019) 분할 및 저장 중...")
test_df = df[df['date'].dt.year > 2000].copy()
test_df.to_parquet(TEST_FILENAME, index=False)
print(f"저장 완료: {TEST_FILENAME} ({len(test_df):,} 레코드)")

훈련/검증 데이터 (1993-2000) 분할 및 저장 중...
저장 완료: data_1993_2000_train_val.parquet (13,832,356 레코드)
테스트 데이터 (2001-2019) 분할 및 저장 중...
저장 완료: data_2001_2019_test.parquet (20,434,824 레코드)
